CS541 Homework 4: Compute the stochastic gradient of the objective function of 1-bit recommendation system

Work Completed by Matthew Halvorsen

This acts as a one bit recommendation system, the data used in not real so the output is a hypothetical log loss for 5 epochs on a binary system. If a real data set was used the test log-loss and accuracy would be used. 

In [ ]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

class OneBitMF:
    def __init__(
        self, n_users, n_items, rank=32, labels="01",
        lambda_u=1e-3, lambda_v=1e-3, lambda_b=1e-4, lambda_c=1e-4,
        lr=0.05, batch_size=4096, n_epochs=10, seed=42
    ):
        """
        Optimizes the objective:  (average logistic loss) + L2 regularization.
        labels: "01" for y∈{0,1}; "pm1" for y∈{-1,+1}
        """
        rng = np.random.default_rng(seed)
        self.U = 0.01 * rng.standard_normal((n_users, rank))
        self.V = 0.01 * rng.standard_normal((n_items, rank))
        self.b = np.zeros(n_users)   # user bias
        self.c = np.zeros(n_items)   # item bias
        self.mu = 0.0                # global bias

        self.rank = rank
        self.labels = labels
        self.lambda_u = lambda_u
        self.lambda_v = lambda_v
        self.lambda_b = lambda_b
        self.lambda_c = lambda_c
        self.lr = lr
        self.batch_size = batch_size
        self.n_epochs = n_epochs

    def _batch_iter(self, I, J, Y, batch_size):
        n = len(Y)
        idx = np.arange(n)
        np.random.shuffle(idx)
        for s in range(0, n, batch_size):
            b = idx[s:s+batch_size]
            yield I[b], J[b], Y[b]

    def fit(self, I, J, Y):
        """
        I, J, Y are 1D arrays of the same length with observed (user i, item j, label y).
        I: user indices in [0, n_users)
        J: item indices in [0, n_items)
        Y: labels (either 0/1 or -1/+1 depending on `labels`)
        """
        n = len(Y)
        for epoch in range(self.n_epochs):
            for Ib, Jb, Yb in self._batch_iter(I, J, Y, self.batch_size):
                # Forward pass (vectorized over the mini-batch)
                z = (self.U[Ib] * self.V[Jb]).sum(axis=1) + self.b[Ib] + self.c[Jb] + self.mu

                if self.labels == "01":
                    p = sigmoid(z)
                    e = p - Yb.astype(np.float64)        # gradient wrt z
                elif self.labels == "pm1":
                    # loss = log(1 + exp(-y*z)); d/dz = - y * sigmoid(-y*z)
                    y = Yb.astype(np.float64)
                    e = - y * sigmoid(-y * z)
                else:
                    raise ValueError("labels must be '01' or 'pm1'")

                m = len(Yb)               # batch size (may be last partial batch)
                scale = 1.0 / m           # we optimize average loss + regularization

                # Accumulate data gradients into full-sized buffers
                gU = np.zeros_like(self.U)
                gV = np.zeros_like(self.V)
                gb = np.zeros_like(self.b)
                gc = np.zeros_like(self.c)
                gmu = e.sum() * scale

                # gU[i] += (e * V[j]); gV[j] += (e * U[i]); gb[i] += e; gc[j] += e
                np.add.at(gU, Ib, (e[:, None] * self.V[Jb]) * scale)
                np.add.at(gV, Jb, (e[:, None] * self.U[Ib]) * scale)
                np.add.at(gb, Ib, e * scale)
                np.add.at(gc, Jb, e * scale)

                # Add L2 regularization gradients (on the rows/entries touched in this batch)
                Ui = np.unique(Ib)
                Vj = np.unique(Jb)
                gU[Ui] += self.lambda_u * self.U[Ui]
                gV[Vj] += self.lambda_v * self.V[Vj]
                gb[Ui] += self.lambda_b * self.b[Ui]
                gc[Vj] += self.lambda_c * self.c[Vj]

                # SGD updates (only on touched indices for efficiency)
                self.U[Ui] -= self.lr * gU[Ui]
                self.V[Vj] -= self.lr * gV[Vj]
                self.b[Ui] -= self.lr * gb[Ui]
                self.c[Vj] -= self.lr * gc[Vj]
                self.mu    -= self.lr * gmu

    # ---- Inference helpers ----
    def score(self, i, j):
        """Raw score z_ij = u_i^T v_j + b_i + c_j + mu."""
        return float(self.U[i] @ self.V[j] + self.b[i] + self.c[j] + self.mu)

    def predict_proba(self, i, j):
        """P(y=1 | i,j) for logistic model."""
        return sigmoid(self.score(i, j))

    def predict(self, i, j, thresh=0.5):
        """Binary prediction in {0,1} using a threshold."""
        return int(self.predict_proba(i, j) >= thresh)


# Minimal working example
if __name__ == "__main__":
    rng = np.random.default_rng(0)
    n_users, n_items, rank = 500, 800, 16

    # Create a small synthetic dataset
    true_U = rng.standard_normal((n_users, rank))
    true_V = rng.standard_normal((n_items, rank))
    true_b = rng.normal(0, 0.2, size=n_users)
    true_c = rng.normal(0, 0.2, size=n_items)
    true_mu = -0.1

    # Sample observations
    N_obs = 100_000
    I = rng.integers(0, n_users, size=N_obs)
    J = rng.integers(0, n_items, size=N_obs)
    z = (true_U[I] * true_V[J]).sum(axis=1) + true_b[I] + true_c[J] + true_mu
    P = sigmoid(z)
    Y = rng.binomial(1, P)  # labels in {0,1}

    # Train
    model = OneBitMF(
        n_users, n_items, rank=rank, labels="01",
        lambda_u=5e-4, lambda_v=5e-4, lambda_b=1e-4, lambda_c=1e-4,
        lr=0.1, batch_size=4096, n_epochs=5, seed=0
    )
    model.fit(I, J, Y)

    # Evaluate on a held-out set (simple AUC-like proxy using probabilities)
    I_te = rng.integers(0, n_users, size=5000)
    J_te = rng.integers(0, n_items, size=5000)
    z_te = (true_U[I_te] * true_V[J_te]).sum(axis=1) + true_b[I_te] + true_c[J_te] + true_mu
    p_te = sigmoid(z_te)
    y_te = rng.binomial(1, p_te)

    pred = np.array([model.predict_proba(i, j) for i, j in zip(I_te, J_te)])
    # Log-loss
    eps = 1e-12
    logloss = -np.mean(y_te * np.log(pred + eps) + (1 - y_te) * np.log(1 - pred + eps))
    # Accuracy at 0.5
    acc = np.mean((pred >= 0.5) == y_te)

    print(f"Test log-loss: {logloss:.4f}   |   Test acc: {acc:.3f}")


Test log-loss: 0.6927   |   Test acc: 0.512
